In [2]:
pip install scikit-learn -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
vod = pd.read_excel('./data/DX데이터스쿨_프로젝트데이터(VOD,커머스,컨텐츠).xlsx', sheet_name='VOD')
commerce = pd.read_excel('./data/DX데이터스쿨_프로젝트데이터(VOD,커머스,컨텐츠).xlsx', sheet_name='커머스')
content = pd.read_excel('./data/DX데이터스쿨_프로젝트데이터(VOD,커머스,컨텐츠).xlsx', sheet_name='콘텐츠')
sampling = pd.read_csv('./data/DX데이터스쿨_프로젝트데이터_channel_pgm_샘플링_3.csv', encoding='cp949')

## 간단한 협업 필터링

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [6]:
# 데이터 불러오기
data=pd.read_excel('./data/DX데이터스쿨_프로젝트데이터(VOD,커머스,컨텐츠).xlsx', sheet_name='VOD')

# 사용자가 시청한 프로그램 정보 추출
user_data = data[data['subsr'] == 66056000]  # 특정 사용자 UUID에 맞게 수정

# 사용자가 시청한 프로그램의 장르 정보 추출
user_genres = user_data['genre_of_ct_cl'].tolist()


### 컨텐츠 기반 필터링

In [ ]:
# TF-IDF 벡터화를 사용하여 프로그램명과 장르를 벡터화
tfidf_vectorizer = TfidfVectorizer()
program_matrix = tfidf_vectorizer.fit_transform(data['asset_nm'] + ' ' + data['genre_of_ct_cl'])

# 사용자가 시청한 프로그램과 유사한 프로그램 찾기
cosine_similarities = linear_kernel(program_matrix, program_matrix)
similar_programs_indices = cosine_similarities[user_data.index[0]].argsort()[:-5:-1]

# 추천 프로그램 출력
recommended_programs = data.loc[similar_programs_indices]
print(recommended_programs[['asset_nm', 'genre_of_ct_cl']])


             asset_nm genre_of_ct_cl
3550  (FREE)롤러코스터(무료)            코미디
3698  (FREE)롤러코스터(무료)            코미디
3697  (FREE)롤러코스터(무료)            코미디
3696  (FREE)롤러코스터(무료)            코미디


### 협업 필터링

In [7]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 불러오기
data = pd.read_excel('./data/DX데이터스쿨_프로젝트데이터(VOD,커머스,컨텐츠).xlsx', sheet_name='VOD')

# 사용자-프로그램 매트릭스 생성
user_program_matrix = data.pivot_table(index='subsr', columns='asset_nm', values='use_tms', fill_value=0)

# 사용자 간의 코사인 유사도 계산
user_similarity = cosine_similarity(user_program_matrix)

# 특정 사용자 UUID
target_user = 66056000

# 특정 사용자의 시청 기록 추출
target_user_history = user_program_matrix.loc[target_user].values.reshape(1, -1)

# 유사한 사용자 찾기
similar_users_indices = user_similarity[user_program_matrix.index == target_user].argsort()[0][::-1][1:]

# 유사한 사용자가 시청한 프로그램 중 사용자가 아직 시청하지 않은 프로그램 추천
recommended_programs = []
for user_index in similar_users_indices:
    similar_user_history = user_program_matrix.iloc[user_index].values.reshape(1, -1)
    unrecommended_programs = similar_user_history - target_user_history

    for idx, value in enumerate(unrecommended_programs[0]):
        if value > 0:
            recommended_programs.append(user_program_matrix.columns[idx])

    if len(recommended_programs) >= 5:
        break

print(recommended_programs)


['(FREE)2067(무료)', '드론-미래전쟁', '무신 : 용의 귀환', '백사 부인', '사일런싱', '스텝업-브레이킹쓰루', '웨이트: 감각에 눈 뜰때', '허트로커', '현장법사-서유기의 시작']
